<a href="https://colab.research.google.com/github/cs16b108/IB_MDDP/blob/master/IB_Diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install webrtcvad

In [71]:
import os
from os.path import isfile, isdir, join
from pathlib import Path
import math
import numpy as np
# import numpy as np
import random
# import math
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt

In [116]:
import collections
import contextlib
import sys
import wave
import webrtcvad


def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])


In [ ]:
def main():
    # if len(args) != 2:
    #     sys.stderr.write(
    #         'Usage: silenceremove.py <aggressiveness> <path to wav file>\n')
    #     sys.exit(1)
    path = '/content/amicorpus/'
    dir_list = sorted(os.listdir(path))
    cnt = 0
    for d in dir_list:
      dir_name = join(path,d)
      if isdir(dir_name):
        filePath = join(path, d, 'audio', d+'.Mix-Headset.wav')
        try:
          audio, sample_rate = read_wave(filePath)
          vad = webrtcvad.Vad(int(1))
          frames = frame_generator(30, audio, sample_rate)
          frames = list(frames)
          segments = vad_collector(sample_rate, 30, 300, vad, frames)

          # Segmenting the Voice audio and save it in list as bytes
          concataudio = [segment for segment in segments]

          joinedaudio = b"".join(concataudio)
          writePath = join('/content/drive/My Drive/amicorpus_non_silence', d, 'audio')
          Path(writePath).mkdir(parents=True, exist_ok=True)
          write_wave(join(writePath, d+'.Mix-Headset.wav'), joinedaudio, sample_rate)
          cnt += 1
          # if(cnt == 2):
          #   break
        except:
          print("Skipping: ", filePath)
    print("Converted: ",cnt)
if __name__ == '__main__':
    main()

In [ ]:
rm -rf amicorpus_non_silence/

In [ ]:
!chmod 755 amiBuild-13720-Mon-Aug-31-2020.wget.sh

In [ ]:
!ls -l '/content/drive/My Drive/amicorpus_non_silence' | wc -l

98


In [ ]:
!./amiBuild-13720-Mon-Aug-31-2020.wget.sh

In [ ]:
!tar -czvf filename.tar.gz amicorpus

In [ ]:
!cp filename.tar.gz /content/drive/My\ Drive/amicorpus.tar.gz

In [ ]:
!tar -xzvf /content/drive/My\ Drive/amicorpus.tar.gz

In [ ]:
join('abc', 'def', 'ncl')

'abc/def/ncl'

In [ ]:
d = 'audio'
d + '.Mix-Headset.wav'

'audio.Mix-Headset.wav'

In [ ]:
!./ComputeFeatures mfcc.config /content/drive/My\ Drive/amicorpus_non_silence/ES2002b/audio/ES2002b.Mix-Headset.wav frameCepstrum+frameDeltaCepstrum sa1.mfcc 0.06 A

In [ ]:
!chmod 755 ComputeFeatures

In [ ]:
!chmod 755 mfcc.config

In [ ]:
pip install python_speech_features
from python_speech_features import mfcc

In [ ]:
import scipy.io.wavfile as wav

In [74]:
overlap = 0.02 #10 ms window shift
fullPath = '/content/drive/My Drive/amicorpus_non_silence/ES2002b/audio/ES2002b.Mix-Headset.wav'
(rate,sig) = wav.read(fullPath)
mfcc_feat = mfcc(sig, rate, numcep = 19, nfilt = 40, winlen=0.03, winstep=overlap)

In [75]:
n, d = mfcc_feat.shape

In [76]:
# overlap = 0.01 #10 ms window shift
init_cluster_time = 2500 #2.5sec
init_cluster_len = math.ceil(init_cluster_time/(overlap*1000))

In [77]:
num_of_clusters = math.ceil(n/init_cluster_len)

In [78]:
num_of_clusters

812

In [108]:
t = np.array_split(mfcc_feat, 3)

In [80]:
class GMM:
    def __init__(self, num_of_clusters):
        self.num_of_clusters = num_of_clusters
        self.log_likelihood =[]
        self.LL_diff = []
        # self.num_of_speakers = num_of_speakers

    def gaussian_prob(self, x, mean, sigma):
        d = x.shape[0]
        p = ((2*math.pi)**(-d/2))*(np.linalg.det(sigma)**(-0.5))*np.exp(-0.5*(x-mean).reshape(d,1).T.dot(np.linalg.inv(sigma)).dot((x-mean).reshape(d,1)))
        return p

    def k_means(self, X):
        n = X.shape[0]
        d = X.shape[1]
        itr = 0
        #self.centroid = np.zeros((self.num_of_clusters, d), dtype = 'float64')
        self.centroids = X[random.sample(range(n), self.num_of_clusters)]
        self.cluster_assigned = np.zeros(n, dtype = int)
        error = 0.0
        while True:
            print("Now at itr - ", itr)
            # print("Centroids - ", self.centroids)
            for i in range(n):
                f_vec = X[i]
                dist = np.sqrt(np.sum((f_vec-self.centroids)**2, 1))
                # print("Dist Shape is - ", dist.shape)
                self.cluster_assigned[i] = np.argmin(dist)
            new_error = np.sum(np.sqrt(np.sum((X - self.centroids[self.cluster_assigned])**2, 1)))
            if(itr>0):
                print("Error Difference is - ", np.abs(error-new_error))
            new_centroids = np.zeros((self.num_of_clusters, d), dtype = 'float64')
            count_of_elements = np.zeros(self.num_of_clusters, dtype = int)
            for i in range(n):
                c_ind = self.cluster_assigned[i]
                new_centroids[c_ind] += X[i]
                count_of_elements[c_ind] += 1
            new_centroids = new_centroids/count_of_elements[:,None]
            if np.abs(new_error-error)<10 or np.array_equal(self.centroids, new_centroids) or itr>=5:
                print("Breaking at itr - ", itr)
                break
            else:
                self.centroids = np.copy(new_centroids)
            itr += 1
            error = new_error

    def EM_GMM_INBUILT(self, X):
        N = X.shape[0]
        d = X.shape[1]
        from sklearn.mixture import GaussianMixture as GMM
        g = GMM(n_components=64, covariance_type = 'full', max_iter = 1)
        g.fit(X)
        print("Created")

    def EM_GMM(self, X):
        N = X.shape[0]
        d = X.shape[1]
        self.cov_mat = np.zeros((self.num_of_clusters, d, d), dtype = 'float64')
        self.gamma = np.zeros((N,self.num_of_clusters), dtype = 'float64')
        likelihood = np.zeros((N,self.num_of_clusters), dtype = 'float64')
        self.pi_prob = np.zeros(self.num_of_clusters, dtype = 'float64')
        self.Nk = np.zeros(self.num_of_clusters, dtype = 'float64')
        for k in range(self.num_of_clusters):
            indices = (np.argwhere(self.cluster_assigned==k)).ravel()
            X_k = X[indices]
            X_k_centered = X_k - self.centroids[k]
            self.Nk[k] = X_k.shape[0]
            # print("Xk ",X_k.shape)
            # print("Xkc ",X_k_centered.shape)
            # print("cov mat ",self.cov_mat[k])
            self.cov_mat[k] = (1/self.Nk[k])*(X_k_centered.T.dot(X_k_centered))
        # print(self.Nk)
        self.pi_prob = self.Nk/N
        print("EM Begins")
        itr = 1
        prev_log_likelihood = 0.0
        
        while True:
            #####################################
            ############   E Step   #############
            #####################################
            for k in range(self.num_of_clusters):
                #self.gamma[i,k] = self.gaussian_prob(X[i], self.centroids[k], self.cov_mat[k])
                self.cov_mat[k] += 1e-6*np.identity(d)
                likelihood[:,k] =  multivariate_normal.pdf(X, self.centroids[k], self.cov_mat[k]).ravel()
                # print("Done ", k)
            # log_likelihood = np.sum(np.sum((likelihood*self.pi_prob), axis = 1))

            # for i in range(N):
            #     print("Done ",i)
             
            self.gamma = likelihood*self.pi_prob
            self.gamma = self.gamma/(np.sum(self.gamma, axis = 1)[:,None])
            # print("E done")

            #####################################
            ############   M Step   #############
            #####################################
            self.Nk = np.sum(self.gamma, axis = 0)
            self.pi_prob = self.Nk/N
            for k in range(self.num_of_clusters):
                self.centroids[k] = (1/self.Nk[k])*np.sum((X*self.gamma[:,k][:,np.newaxis]), axis = 0)
                X_centered = X - self.centroids[k]
                self.cov_mat[k] = (1/self.Nk[k])*((X_centered*self.gamma[:,k][:,np.newaxis]).T.dot(X_centered))
            # print("M done")

            #####################################
            ########   Log Likelihood   #########
            #####################################
            new_log_likelihood = np.sum(np.log(np.sum((likelihood*self.pi_prob), axis = 1)))
            self.log_likelihood.append(new_log_likelihood)
            diff_LL = np.abs(new_log_likelihood-prev_log_likelihood)
            self.LL_diff.append(diff_LL)
            print("Itr = ", itr, " Current LL is - ",new_log_likelihood)
            print("Change In LL is - ",diff_LL)
            if(diff_LL<100 or itr>=10):
                print("EM Finished at iteration - ", itr)
                break
            itr += 1
            prev_log_likelihood = new_log_likelihood

In [81]:
ug = GMM(num_of_clusters)
ug.k_means(mfcc_feat)
ug.EM_GMM(mfcc_feat)

Now at itr -  0
Now at itr -  1
Error Difference is -  376247.70135345636
Now at itr -  2
Error Difference is -  67175.10419689165
Now at itr -  3
Error Difference is -  28711.851844280027
Now at itr -  4
Error Difference is -  15926.86318728514
Now at itr -  5
Error Difference is -  10563.804716072511
Breaking at itr -  5
EM Begins
Itr =  1  Current LL is -  -6763974.208030206
Change In LL is -  6763974.208030206
Itr =  2  Current LL is -  -6719287.087322179
Change In LL is -  44687.12070802692
Itr =  3  Current LL is -  -6695338.646181228
Change In LL is -  23948.44114095159
Itr =  4  Current LL is -  -6679947.446385758
Change In LL is -  15391.199795469642
Itr =  5  Current LL is -  -6668993.752974289
Change In LL is -  10953.69341146946
Itr =  6  Current LL is -  -6661653.686079189
Change In LL is -  7340.066895099357
Itr =  7  Current LL is -  -6656149.916194153
Change In LL is -  5503.769885036163
Itr =  8  Current LL is -  -6651833.96698823
Change In LL is -  4315.949205922894
I

In [111]:
def calc_prob(x, ug):
  p = 0.0
  D = x.shape[0]
  for i in range(D):
    s = x[i]
    for k in range(ug.num_of_clusters):
    #self.gamma[i,k] = self.gaussian_prob(X[i], self.centroids[k], self.cov_mat[k])
      cov_matrix = 1e-6*np.identity(d) + ug.cov_mat[k]
      # cov_matrix = 
      p =  p + ug.pi_prob[k]*multivariate_normal.pdf(s, ug.centroids[k], cov_matrix)
  p = p/D
  return p

In [114]:
########################
##### IB Algorithm #####
########################

#Init Variables
N = num_of_clusters
C = np.array_split(mfcc_feat, num_of_clusters)
prob_c = np.zeros(N, dtype = float)
prob_cond_y_c = np.zeros((N, N), dtype = float)
prob_cond_c_x = np.zeros((N, N), dtype = float)
del_F = np.zeros((N, N), dtype = float)
for i in range(N):
  prob_c(i) = calc_prob(C[i], ug)
  for j in range(N):
    prob_cond_y_c[j][i] = calc_cond_prob(j, C[i], ug)
    if(j == i):
      prob_cond_c_x[j][i] = 1

for i in range(N):
  for j in range(i+1, N): 
    del_F[i][j] = cal_objective_diff(C[i], C[j])

#Main Algo





